In [3]:
import numpy as np
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

In [6]:
from datasets import load_dataset

ds = load_dataset("aaronjpi/fruits-vegetables-price")

In [7]:
print("Dataset structure:")
print(ds)

# Most datasets have a 'train' split, so assuming that exists:
if 'train' in ds:
    # Display the first 5 rows of the train split
    print("\nFirst 5 rows of the training set:")
    for i, example in enumerate(ds['train']):
        if i < 5:
            print(f"Example {i+1}:")
            print(example)
        else:
            break
else:
    # If there's no 'train' split, check what splits are available and show the first one
    first_split = list(ds.keys())[0]
    print(f"\nFirst 5 rows of the '{first_split}' split:")
    for i, example in enumerate(ds[first_split]):
        if i < 5:
            print(f"Example {i+1}:")
            print(example)
        else:
            break

# Alternative method using slicing (more concise)
print("\nFirst 5 rows using slicing:")
if 'train' in ds:
    print(ds['train'][:5])
else:
    first_split = list(ds.keys())[0]
    print(ds[first_split][:5])

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['productname', 'date', 'farmprice', 'atlantaretail', 'chicagoretail', 'losangelesretail', 'newyorkretail', 'averagespread'],
        num_rows: 15766
    })
})

First 5 rows of the training set:
Example 1:
{'productname': 'Strawberries', 'date': datetime.datetime(2019, 5, 19, 0, 0), 'farmprice': '$1.16', 'atlantaretail': '$2.23', 'chicagoretail': '$1.70', 'losangelesretail': '$1.99', 'newyorkretail': '$2.54', 'averagespread': '82.33%'}
Example 2:
{'productname': 'Romaine Lettuce', 'date': datetime.datetime(2019, 5, 19, 0, 0), 'farmprice': '$0.35', 'atlantaretail': '$1.72', 'chicagoretail': '$2.00', 'losangelesretail': '$1.69', 'newyorkretail': '$1.99', 'averagespread': '428.57%'}
Example 3:
{'productname': 'Red Leaf Lettuce', 'date': datetime.datetime(2019, 5, 19, 0, 0), 'farmprice': '$0.32', 'atlantaretail': '$1.84', 'chicagoretail': '$1.84', 'losangelesretail': '$1.69', 'newyorkretail': '$1.89', 'averagespread': 